# Astronomy 8824 - Problem Set 1

The goal of this problem set is to write routines to perform various numerical integrals

In [1]:
import math
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# matplotlib settings 
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)   # fontsize of the figure title

The goal of this problem set is to write a program to do numerical integrals of user specified
functions, and to compare the performance of some simple algorithms for doing so. I recommend that you read sections 4.0-4.4 of Numerical Recipes as background. However, while their description of algorithms and their properties is good, you should write your own code rather than borrow the NR subroutines.

Specifically, you will write a Python program that computes

$$
I = \int^{a}_{b} f(x) dx
$$ 

where a and b are finite limits of integration and we will use various choices for f(x) below.
Start with the code below, which implements the “Euler method”

$$
I = \sum^{N}_{i=1} f(x_i) h_n
$$

where N is the number of (equal-sized) integration steps and

$$
h_N = \frac{b-a}{N}, x_i = a + (i-1) h_N
$$

In [2]:
# Routines from DHW
# Minor modifications by PM to output info about interations

def integrate_driver(func,integrator,a,b,tolerance=1.e-4,nstepmax=2.e8,verbose=False):
    """
    Integrate a function func() using the specified integrator routine
    integrator = euler, euler_loop, trapzd, or midpoint
    a = lower limit of integration
    b = upper limit of integration
    tolerance = fractional convergence required for integral, default 1.e-4
    nstepmax = maximum number of steps allowed, default 2e8
    verbose = 1 -> write individual iterations to "iterations.out", default True

    Number of steps starts at 4 and doubles until convergence or nstep>nstepmax
    """

    if (verbose):
        f=open("iterations.out","w")
        nsteps = []
        hsteps = []
        integrals = []
    nstep=4
    oldint=0.0    
    integral=integrator(func,a,b,nstep)

    while ((np.fabs(oldint/integral-1.0) > tolerance) and (2*nstep<nstepmax)):
        oldint=integral
        nstep*=2
        integral=integrator(func,a,b,nstep)
        if (verbose):
            hstep=(b-a)/nstep
            outstring="%8d %.8g %.8g\n" % (nstep,hstep,integral)
            nsteps.append(nstep)
            hsteps.append(hstep)
            integrals.append(integral)
            f.write(outstring)

    if (verbose):
        f.close()
        return [integral, nstep, nsteps, hsteps, integrals]
    if (np.fabs(oldint/integral-1.0) > tolerance):
        print("Warning, fractional convergence is only ", np.fabs(oldint/integral-1.0), " with ", integrator)
    return [integral, nstep]

def euler_loop(func,a,b,nstep):
    """
    Evaluate [\int_a^b func(x) dx] using Euler rule with nstep steps
    Use loop analogous to C or fortran
    """
    hstep=(b-a)/nstep
    y=a                                
    integral=hstep*func(y)
    for i in range(nstep-1):  # no more xrange in Python 3
        y+=hstep
        integral+=func(y)*hstep
    return(integral)

def euler(func,a,b,nstep):
    """ 
    Evaluate [\int_a^b func(x) dx] using Euler rule with nstep steps
    Use numpy array operations
    """
    hstep=(b-a)/nstep
    x=np.linspace(a,b-hstep,nstep)
    y=func(x)*hstep
    return (np.sum(y))

## 1. Speed, Convergence, and Euler

Look through the code so that you understand its structure. Note in particular that it
automatically doubles the number of steps (starting at N = 4) until it converges to a specified
fractional tolerance. The program calculates the integral with N = 4, then doubles the number
of steps and compares the answers. If the fractional difference is larger than the tolerance, it
doubles the number of steps again, continuing until the fractional difference between two
successive evaluations is less than the tolerance. It also has a safeguard with a maximum
number of steps, to prevent the program from running forever if it doesn’t converge.
Using this code, compute the integral

$$
\int^{5}_{1} \frac{1}{x^{3/2}} dx
$$

How many steps (approximately) are required to get an answer with a fractional error $|(I
−I_{exact})/I_{exact}| < 10^{−4}$? $10^{-6}$? 

In [3]:
### Answer

For a specified number of steps, the integral is evaluated using either the function
euler_loop or euler. The former uses a loop structure typical of fortran or C, while the
latter uses array operations available in NumPy.

Compare the speed of these two implementations using the %timeit function of iPython.

In [4]:
### Answer

Note the substantial speedup with the numpy array approach!

## 2.  Trapezoidal Rule

Modify the code so that it implements the "Trapezoidal Rule": 

$$
I = \sum^{N}_{i=1} \left[ \frac{1}{2} f(x_i) + \frac{1}{2} f(x_{i+1}) \right] h_N
$$

(This seems at first glance to require twice as many function evaluations as the Euler method,
but there is an obvious way to avoid this, which you should implement in your program.)

You can modify either the euler or euler_loop function to accomplish this, or you can
start your own code from scratch. If you write your own code, you
should maintain the automatic step-doubling-to-convergence feature.

Numerically compute the integral from Part 1 with both methods. How many steps
(approximately) are required to get an answer with a fractional error $|(I−I_{exact})/I_{exact}| < 10^{−4}$ with the Trapezoidal Rule? $10^{-6}$? 

In [5]:
### Answer

## 3. Simpson's Rule

With step doubling, you can implement a neat trick, described in Numerical Recipes. Given
estimates $IT_N$ and $IT_{N/2}$ from the Trapezoidal Rule using N and N/2 steps, make the new
estimate 

$$
IS = (4 IT_N − IT_{N/2})/3
$$. 

This approximation, Simpson’s Rule, should converge faster
than the Trapezoidal Rule. Write a routine to compute an integral via Simpson’s Rule using
this trick. This involves changing the integration driver routine; for any given N you are still
using the Trapezoidal Rule.

Make a plot of the error $|(I−I_{exact})/I_{exact}|$ vs. the step size h for the Euler, Trapezoidal, and
Simpson’s Rule evaluations of the above numerical integral. Include this plot (log-log is recommended) as part of your solution set. Is the behavior what you expect?

Note: If Simpson’s doesn’t converge noticeably faster than the Trapezoidal Rule, there is a
bug in your program.

In [6]:
### Answer 

In [7]:
### Answer

## 4. Solve some integrals with these methods

For each of the following integrals, give the value of the integral I for each of the three
numerical integration methods and the number of steps needed to get convergence to a
fractional tolerance of $10^{−6}$.

1. 
$$
\int^{2}_{1} \frac{1}{x^{3/2} (1 + x^{3/2})} dx
$$

2. 
$$
\int^{100}_{1} \frac{{\rm sin} x}{x} dx
$$

3. 
$$
\int^{1000}_{1} \frac{ {\rm sin^2} x}{x^2} dx
$$

4. 
$$
\int^{1000}_{1} \left( x + \frac{1}{x} \right)^{-1} dx
$$

5. 
$$
\int^{{\rm ln} 1000}_{0} \left( 1 + e^{-2x} \right)^{-1} dx
$$

Note: If you are using an array-based implementation, you will need to use np.sin in your integrand, e.g., return (np.sin(x)/x). If you are using a loop-based implementation, you can use either np.sin or math.sin. Do you know why?

In [8]:
### Answer

## 5. Midpoint Method

Write a routine that implements the Midpoint Rule: 

$$
I = \sum^{N}_{i=1} f(x_{i+1/2}) h_N
$$
where $x_{i+1/2} = x_i + h_N/2$. 

For the test integral of Question 1, compare the convergence of the Midpoint Method to that of the
Euler, Trapezoid, and Simpson’s Rule methods. Add the Midpoint results to your
convergence plot.

Use your routine to compute the integrals: 

6. 
$$
\int^{4}_{0} \frac{1}{x^{1/2}} dx
$$
7. 
$$
\int^{\pi}_{0} \frac{{\rm sin} x}{x} dx
$$

Why is the Midpoint Rule useful even though it is less accurate than Simpson’s Rule for the
same number of steps? Why does the second integral converge much faster than the first
integral?

In [9]:
### Answer 

## 6. More tricky integrals

Numerically compute the integrals: 

8. 
$$
\int^{\infty}_{1} \frac{1}{x^2 + x^3} dx
$$
9. 
$$
\int^{\infty}_{1} \frac{{\rm sin^2} x}{x^2} dx
$$

Explain how you did it. (Hint: See NR Sec 4.4, or the lecture notes. You do not need to
approximate $\infty$ by a large finite number.)

Getting an accurate result for the second integral is much harder than for the first integral. Why?

In [12]:
### Answer

## 7. (Optional) Comparison to scipy.integrate

There are numerical integration routines available in scipy.integrate. You can see what they are with 

In [13]:
import scipy.integrate as si

and get more information with: 

In [14]:
si?

Type:        module
String form: <module 'scipy.integrate' from '/apps/python/3.7-2019.10/lib/python3.7/site-packages/scipy/integrate/__init__.py'>
File:        /apps/python/3.7-2019.10/lib/python3.7/site-packages/scipy/integrate/__init__.py
Docstring:  
Integration and ODEs (:mod:`scipy.integrate`)

.. currentmodule:: scipy.integrate

Integrating functions, given function object

.. autosummary::
   :toctree: generated/

   quad          -- General purpose integration
   dblquad       -- General purpose double integration
   tplquad       -- General purpose triple integration
   nquad         -- General purpose n-dimensional integration
   fixed_quad    -- Integrate func(x) using Gaussian quadrature of order n
   quadrature    -- Integrate with given tolerance using Gaussian quadrature
   romberg       -- Integrate func using Romberg integration
   quad_explain  -- Print information for use of quad
   newton_cotes  -- Weights and error coefficient for Newton-Cotes integration
   Integ

And explore individual routines, e.g. with "si.quad?".

For the 3rd and 4th integrals (Problem 4, the ones with upper limits of 1000), use the scipy routines
si.quad and si.romberg to evaluate the integrals. Do you get the same answers that you got from Simpson’s Rule? Use %timeit in iPython to compare their speed to that of your Simpson’s Rule program for these two integrals. What do you find? 

In [15]:
### Answer 

## 8. Comoving Distance

So that you end this exercise with something of practical use (and we’ll use it later in the
semester), adapt your Simpson’s Rule integrator to a program that specifically computes the
co-moving distance to an object at redshift z in a flat universe with a cosmological constant.
The formula for the comoving distance is

$$
D_C(z) = \frac{c}{H_0} \int^{z}_{0} \frac{H_0}{H(z')} dz'
$$
with
$$
\frac{H(z)}{H_0} = \left[ \Omega_m (1 + z)^3 + \Omega_\Lambda \right]^{1/2} 
$$

Your program should take as arguments $\Omega_m$, $H_0$ (in km s$^{−1}$ Mpc$^{−1}$), and z. Because we are assuming a flat universe, $\Omega_\Lambda = 1 - \Omega_m$. 

For $\Omega_m = 0.3$ and $H_0 = 67$ km s$^{−1}$ Mpc$^{−1}$, what is the comoving distance to redshifts 0.5, 1, and 2 (in Mpc)?

(Optional) compare your result with astropy.cosmology

In [16]:
### Answer 